#  Populate MongoDB Atlas Database
In this Python notebook, we will be generating embeddings for two 10K filing documents, indexing them and storing them into MongoDB Atlas database.

## Basic Setup
We'll start with some basic setup steps in the next two code cells. Don't worry if your device does not have a GPU, you will still be able to proceed with the Quest.

In [1]:
# Check if GPU is enabled
import os
import torch

# To disable GPU and experiment, uncomment the following line
# Normally, you would want to use GPU, if one is available
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  False


In [2]:
# Setup logging. To see more logging, set the level to DEBUG
import sys
import logging

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Step 1: Inspecting the Documents

As mentioned earlier in the quest, we'll be using two 10K filings as our corpus. These are financial documents filed by US public companies to SEC (Securities and Exchange Commission). You can read about them [here](https://www.investor.gov/introduction-investing/investing-basics/glossary/form-10-k).

The two 10K filings we'll be using are Lyft and Uber. You can find these documents in the `data` folder:
```text
data/10k
├── lyft_2021.pdf
└── uber_2021.pdf
```

Don't be fooled by the fact that it is just 2 documents - each PDF document is over 200+ pages long! These aren't any ordinary simple text documents, these are very dense text documents containing a lot of very specific information. 

Go ahead and inspect these documents manually before proceeding with this notebook to get a feel for the kinds of documents we're dealing with.

## Step 2: Load Settings

In [3]:
# Load settings from .env file
from dotenv import find_dotenv, dotenv_values

# Change system path to root direcotry
sys.path.insert(0, '../')

# _ = load_dotenv(find_dotenv()) # read local .env file
config = dotenv_values(find_dotenv())

# For debugging purposes
# print (config)

ATLAS_URI = config.get('ATLAS_URI')

if not ATLAS_URI:
    raise Exception ("'ATLAS_URI' is not set.  Please set it above to continue...")
else:
    print("ATLAS_URI Connection string found")

## Only uncomment this if you are using OpenAI for embeddings
# OPENAI_API_KEY = config.get("OPENAI_API_KEY")
# if not OPENAI_API_KEY:
#     raise Exception ("'OPENAI_API_KEY' is not set. Please set it above to continue...")
# else:
#     print("ATLAS_URI Connection string found:", ATLAS_URI)

ATLAS_URI Connection string found


In [4]:
# Define our variables
DB_NAME = 'abdulabiola21'
COLLECTION_NAME = 'ele-562'
INDEX_NAME = 'lambda_embedding'

In [5]:
# LlamaIndex will download embeddings models as needed
# Set llamaindex cache dir to ../cache dir here (Default is system tmp)
# This way, we can easily see downloaded artifacts
os.environ['LLAMA_INDEX_CACHE_DIR'] = os.path.join(os.path.abspath('../'), 'cache')

In [6]:
import pymongo

mongodb_client = pymongo.MongoClient(ATLAS_URI)

print ("Atlas client initialized")

Atlas client initialized


## Step 3: Clear out the collection

In the event you're repeating this Quest again, run the code cell below for a fresh start! This code cell will delete the documents you've previously added to the `rag1 - 10k` collection (if any). If you are running this Quest for the first time, you should expect to see `0` below because you've yet to add any items into this collection yet.

In [7]:
database = mongodb_client[DB_NAME]
collection = database [COLLECTION_NAME]

doc_count = collection.count_documents (filter = {})
print (f"Document count before delete : {doc_count:,}")

result = collection.delete_many(filter= {})
print (f"Deleted docs : {result.deleted_count}")

INFO:pymongo.serverSelection:{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "count", "topologyDescription": "<TopologyDescription id: 6639485a892cc1eeb9780eeb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-rtbkmtd-shard-00-00.mmc8nho.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-rtbkmtd-shard-00-01.mmc8nho.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-rtbkmtd-shard-00-02.mmc8nho.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6639485a892cc1eeb9780eeb"}, "remainingTimeMS": 29}
{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "count", "topologyDescription": "<TopologyDescription id: 6639485a892cc1eeb9780eeb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-rtbkmtd-shard-00-00.mmc8nho.mongodb.net', 27017) server_type: Unknown, rtt: None>, <Serv

## Step 4: Setup Embeddings

Now, we'll need to establish an embedding model to help us generate embeddings for the documents we'll be uploading. Here, we provide two options - the first option is to use an OpenAI embedding model and the second option is to use an open source HuggingFace embedding model. We'll be going with the second approach here.

### 4.1 - Option B : Using Custom Embeddings

This option utilizes a HuggingFace embedding model. Recap from Quests 3 and 4 that you can choose from a large number of options. Listed below are some examples, taken from the leaderboard https://huggingface.co/spaces/mteb/leaderboard. We'll be going with the `BAAI/bge-small-en-v.15` embedding model here.

| model name                              | overall score | model size | model params | embedding length | License  | url                                                            |
|-----------------------------------------|---------------|------------|--------------|------------------|----------|----------------------------------------------------------------|
| BAAI/bge-large-en-v1.5                  | 64.x          | 1.34 GB    | 335 M        | 1024             | MIT      | https://huggingface.co/BAAI/bge-large-en-v1.5                  |
| BAAI/bge-small-en-v1.5                  | 62.x          | 133 MB     | 33.5 M       | 384              | MIT      | https://huggingface.co/BAAI/bge-small-en-v1.5                  |
| sentence-transformers/all-mpnet-base-v2 | 57.8          | 438 MB     |              | 768              | Apache 2 | https://huggingface.co/sentence-transformers/all-mpnet-base-v2 |
| sentence-transformers/all-MiniLM-L12-v2 | 56.x          | 134 MB     |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2 |
| sentence-transformers/all-MiniLM-L6-v2  | 56.x          | 91 MB      |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2  |

In [8]:
# from llama_index.embeddings import HuggingFaceEmbedding
# Uncomment the line above and comment the line below if you face an import error
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
NumExpr defaulting to 4 threads.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [9]:
## Set up embedding model
# The LLM used to generate natural language responses to queries
# If not provided, it will default to gpt-3.5-turbo from OpenAI
# If your OpenAI API key is not set, it will default to llama2-chat-13B from Llama.cpp
# Since we don't need an LLM just yet, we'll be setting it to None

# from llama_index import ServiceContext
# Uncomment the line above and comment away the line below if you face an import error
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

/tmp/ipykernel_74881/3880219279.py:11: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


LLM is explicitly disabled. Using MockLLM.


## Step 5: Connect Llama-Index and MongoDB Atlas

As mentioned in the Quest, we'll be using MongoDB Atlas as our vector storage. This is critical to store indexed data and then query later on.

In [10]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
# from llama_index.storage.storage_context import StorageContext
# Uncomment the line above and comment away the line below if you face an import error
from llama_index.core import StorageContext

vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                 db_name = DB_NAME, collection_name = COLLECTION_NAME,
                                 index_name  = INDEX_NAME,
                                 ## The following columns are set to default values
                                 # embedding_key = 'embedding', text_key = 'text', metadata_= 'metadata',
                                 )

storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Step 6: Read PDF Documents

Llama-index has very handy `SimpleDirectoryReader` that can read single/multiple files and also an entire directory's content. We'll be using this to read our 2 PDF files and storing the data in `docs`.

In [12]:
%%time

# from llama_index.readers.file.base import SimpleDirectoryReader
# Uncomment the line above and comment away the line below if you face an import error
from llama_index.core import SimpleDirectoryReader

data_dir = '../data/ELE562'

docs = SimpleDirectoryReader(
        input_dir=data_dir
).load_data()

print (f"Loaded {len(docs)} chunks from '{data_dir}'")

Loaded 128 chunks from '../data/ELE562'
CPU times: user 11.8 s, sys: 87.7 ms, total: 11.8 s
Wall time: 11.9 s


## Step 7: Index the Documents and Store Them Into MongoDB Atlas

The code cell below is where everything that we've been preparing for in this Python notebook comes together:
- Embeddings are generated using our packaged-up embedding model `service_context` 
- Our documents `docs` get indexed `storage_context` - both text and embeddings are stored in MongoDB Atlas

In [13]:
%%time

# from llama_index.indices.vector_store.base import VectorStoreIndex
# Uncomment the line above and comment away the line below if you face an import error
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs, 
    storage_context=storage_context,
    service_context=service_context,
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1min 41s, sys: 45.9 s, total: 2min 27s
Wall time: 1min 19s
